# Algoritmo Click
## Generacion de cliques
+ Se generan con biopandas para obtener los atomos de  CαCα  y sus coordenadas.
+ Se calcula la distancia y se genera un grafo completo con la distancia entre cada par de atomos.
+ Se restringen los enlaces por una distancia dada y se generan los cliques que tengas un numero k de elementos para pertencer al clique.
+ Una ves generados los cliques de cada proteina se extraen sus coordenadas para poderlas comparar

In [3]:
#libreria de analisis de datos y una caracterizacion para su facil lectura.
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 40)
pd.set_option('display.max_colwidth', -1)
#libreria de generacion de rede y cliques
import networkx as nx,community

#libreria de visualizacion de datos y un formato dado
import matplotlib.pyplot as plt
plt.style.use('ggplot')
font = {'family' : 'sans',
        'weight' : 'bold',
        'size'   : 20}
plt.rc('font', **font)
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams[u'figure.figsize'] = (16,8)

#mas librerias que voy obteniendo
import biopandas.pdb as bp
biop = bp.PandasPdb() #libreria de lectura de pdbs

#libreria de calculo de distancia euclidiana
from scipy.spatial.distance import pdist, squareform

#libreria de mate
import numpy as np

#libreria de iteraciones
import itertools as it

#Libreria de MA para RMSD
import sys
sys.path.append('math_tricks/')
import math_vect_tools as mvt

#libreria para correr dssp desde bash
import subprocess as sp

#libreria para Parsear DSSP FIles
import DSSPData as dd

#Libreria de graficacion interactiva
# import plotly.plotly as py
# import plotly.graph_objs as go

In [4]:
# Aqui se cambiaria por los archivos a leer pdbs sin modificar
path1 = '1xxa.pdb'
path2 = '1tig.pdb'

#funcion de lectura con biopandas
def read_biopdb(path):
    """Extrae las cordenadas de los atomos de C_alfa y los acomoda en un vector
    devuelve un dataframe con las coordenadas y el numero de residuo"""
    df = biop.read_pdb(path)
    df_atom = df.df['ATOM']
    df_atom = df_atom[df_atom.chain_id == 'A']
    #OJO AQUI ESTA ADECUADO AL PDB   para elegir solo un frame en trj_0 y trj_0_A [:1805]
    df_ca = df_atom[[
        'atom_number', 'atom_name', 'residue_name', 'residue_number',
        'x_coord', 'y_coord', 'z_coord'
    ]]
    
    columna_vector = []
    for i in zip(df_ca.x_coord.tolist(), df_ca.y_coord.tolist(),
                 df_ca.z_coord.tolist()):
        columna_vector.append(np.array(i))

    df_ca['vector'] = columna_vector
    return (df_ca)

In [5]:
#lectura de pdbs
df_ca1 = read_biopdb(path1)
df_ca2 = read_biopdb(path2)

/home/serch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
df_ca1.head(5)

,atom_number,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,vector
0,1,N,LEU,82,28.39000,57.09300,26.05600,"[28.39, 57.093, 26.056]"
1,2,CA,LEU,82,28.95200,56.59300,24.81300,"[28.952, 56.593, 24.813]"
2,3,C,LEU,82,28.02100,57.00900,23.67800,"[28.021, 57.009, 23.678]"
3,4,O,LEU,82,26.79900,56.96700,23.81800,"[26.799, 56.967, 23.818]"
4,5,CB,LEU,82,29.10200,55.06200,24.87700,"[29.102, 55.062, 24.877]"


In [7]:
#se calcula la distancia entre cada par de nodos.
def distancia_entre_atomos(df_ca):
    """df_ca: Dataframe con coordenadas de los atomos alfa, devuelve otro DataFrame
    df_da: Dataframe como una matriz de adyacencias donde el valor es la distancia"""
    df_ca = df_ca[df_ca.atom_name == 'CA']
    distancias = []
    #se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v,i in zip(df_ca.vector,df_ca.atom_number):
        distancia_un_atomo = []
        for av,j in zip(df_ca.vector,df_ca.atom_number):
            distancia = pdist([v,av],metric='euclidean').item()
            distancia_un_atomo.append(distancia)
        distancias.append(distancia_un_atomo)
    #se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=df_ca.atom_number,columns=df_ca.atom_number,
                         data=distancias)
    return(df_da)

In [8]:
#generacion de matriz de adyacencias
df_da1 = distancia_entre_atomos(df_ca1)
df_da2 = distancia_entre_atomos(df_ca2)
#podriamos solo mantener la matriz diagonal y dejarla como un array de arrays

In [9]:
df_da1.head()

atom_number,2,10,19,27,35,42,50,58,66,74,86,94,102,107,114,121,128,136,146,153,...,376,383,390,397,402,410,414,425,432,439,448,456,464,476,485,490,498,506,515,523
atom_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.00000,3.81098,5.60378,7.07272,9.15829,12.26662,14.68948,16.21418,19.99722,23.00051,26.67134,29.33216,28.05956,24.28884,20.90471,19.12459,15.41605,15.15388,12.28380,11.90989,...,20.83170,24.04410,25.21675,27.56573,31.20085,31.86499,28.47283,28.28916,25.50663,25.02433,24.42026,21.96133,20.19342,19.93695,18.82373,15.72461,15.36252,16.30178,13.80875,10.68698
10,3.81098,0.00000,3.81358,4.94839,5.69833,8.93145,10.98528,12.41911,16.19648,19.25079,22.94445,25.69477,24.58624,20.85520,17.36409,15.58805,11.89764,11.86866,9.51179,10.02573,...,17.79363,20.96394,22.22639,24.86978,28.42454,28.97565,25.48445,24.99717,22.06621,21.53352,21.22542,18.79927,16.74314,16.72532,15.97746,12.65762,12.18978,13.75620,11.61392,8.03032
19,5.60378,3.81358,0.00000,3.79477,5.48247,7.37654,10.38934,12.89045,16.47232,19.88705,23.55042,26.65060,25.75188,22.10248,18.58456,16.24364,12.51352,11.41885,8.95007,8.40388,...,19.65269,22.94983,24.55698,27.48609,30.96810,31.47594,27.91046,26.98348,23.78842,23.14092,23.38156,21.09070,18.56636,18.79135,18.66005,15.23623,14.35056,16.43023,14.83490,11.10964
27,7.07272,4.94839,3.79477,0.00000,3.81335,6.32807,9.43662,11.79561,15.33944,18.47719,21.96326,24.93128,23.61797,19.91315,16.67197,13.97432,10.22450,8.76053,5.62719,5.12885,...,17.22565,20.77282,22.81751,25.70010,29.26562,30.16803,26.76277,25.82154,22.48060,22.42535,22.78923,20.06255,17.79888,18.72738,18.33605,14.64119,14.62905,17.05988,15.09881,11.38553
35,9.15829,5.69833,5.48247,3.81335,0.00000,3.81005,5.92404,8.01531,11.62045,14.81550,18.38845,21.41318,20.35939,16.71133,13.27971,10.77843,7.03637,6.31148,4.68473,6.51554,...,14.51680,17.85200,19.78998,22.91500,26.33072,27.00376,23.51277,22.33484,18.94227,18.73821,19.31898,16.74853,14.22113,15.29910,15.29917,11.60644,11.45513,14.34638,12.90302,9.23634


In [10]:
df_da1.shape

(71, 71)

In [11]:
def gen_3_cliques(df_da, dth = 10, k=3):
    """Genera n-cliques de dataframe de distancias, tomando en cuenta los enlaces menores o iguales
    a dth y forma los k-cliques que elijas 
    valores por default:
    dth=10, k=3"""
    #red de distancias completa
    red = nx.from_pandas_adjacency(df_da)
#     print("red antes de filtros:",nx.info(red))

    #filtro de distancias
    edgesstrong = [(u,v) for (u,v,d) in red.edges(data=True) if d["weight"] <= dth]

    red = nx.Graph(edgesstrong)
#     print("=="*20)
#     print("red despues de filtros:",nx.info(red))

    n_cliques = [clq for clq in nx.find_cliques(red) if len(clq) >=k]
    print('numero de cliques maximos encontrados:',len(n_cliques))
#     print(n_cliques)

    lista_cliques = []
    for i,v in enumerate(n_cliques):
        a = list(it.combinations(v,k))
        for j in a:
            if set(j) not in lista_cliques:
                #recuerda que para comparar elementos utiliza set, y apilalos como set
                lista_cliques.append(set(j))

    df_lc = pd.DataFrame(lista_cliques)            
    print("numero de %s-cliques posibles:" % (k), df_lc.shape[0])
    return(df_lc,n_cliques)

In [12]:
df_lc1, cliques1 = gen_3_cliques(df_da1,dth = 10, k=3)
print('--'*59)
df_lc2, cliques2 = gen_3_cliques(df_da2,dth = 10, k=3)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
----------------------------------------------------------------------------------------------------------------------
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102


In [13]:
df_lc1.head()
# df_lc1

,0,1,2
0,27,2,35
1,10,2,35
2,19,2,35
3,10,2,27
4,19,2,27


## Calculo del SSM
Para calcular la estructura secundaria, es necesario obtener los angulos dihedrales Phi y Psi, y posteriormente empalmarlo con el diagrama de Ramachandran y observar en que clasificacion cae, donde para fines practicos solo se utilizaran 3 estructuras:
    + alfa helices
    + beta laminas
    + coil cualquier otra estructura no definida
    
Para obtener C, $\alpha$, $\beta$ con:
   + $\Phi$
   + $\Psi$
1. Matriz de comparacion de Estructura Secundaria (SSM)
2. Solvente Accesible (SAM)

### DSSP
desde bash y se extrae la estructura secundaria.

__Que siempre si se utiliza este...__

Para comparar resultados se utiliza este !!!

Diccionario DSSP
<img src='ss.jpeg'/>

In [14]:
def mini_dssp(path,df_ca):
    #ejecuto dssp desde bash y guardo archivo como output.log
    sp.run(['dssp','-i',path,'-o','output.log'])
    #parseo el dssp file
    dd_ob = dd.DSSPData()
    dssp_file_name = open('output.log')
    dd_ob.parseDSSP( 'output.log' )
    #obtengo la estructura y la guardo, posible no es necesario los residuos
    #solo el numero de atomo que le pego arbitrariamente REVISAR si esta bien
    ss = [i[2] for i in dd_ob.struct ]
    ss = pd.DataFrame([i for i in zip(ss,dd_ob.resnum,dd_ob.getChainType())])
    ss.columns = ['pre_ss','residue_number','chain']
    ss = ss[ss.chain == 'A']
    ss = ss[ss.residue_number != ''].reset_index(drop=True)
    ss['atom_number'] = df_ca[df_ca.atom_name == 'CA'].atom_number.values
    #catalogo  Yo tomo B y E como betas, G H I como alfa y lo demás como coil 
    #B - betas 
    #H - alfas
    ss['structure'] = np.where(ss.pre_ss.isin(['B','E']),'B',
                               np.where(ss.pre_ss.isin(['G','H','I']),'H',
                                        'C'))
    #checks
    print(ss.structure.value_counts(normalize = True) * 100)
    print(path)
    return(ss)

In [15]:
ss1 = mini_dssp(path1,df_ca1)
print('//'*40)
ss2 = mini_dssp(path2,df_ca2)

C   43.66197
B   32.39437
H   23.94366
Name: structure, dtype: float64
1xxa.pdb
////////////////////////////////////////////////////////////////////////////////
H   35.22727
B   35.22727
C   29.54545
Name: structure, dtype: float64
1tig.pdb


In [16]:
#funcion para obtener las coordenadas del clique
def get_SS(ss,df_lc):
    """
    """
    #lista para apilar las estructuras
    c1 = []
    c2 = []
    c3 = []

    for i in df_lc.index:
        #si coincide el numero de atomo con el numero de atomo del clique le coloca el vector de coordenadas
        c1_temp = ss[ss.atom_number==df_lc.iloc[i,0]].structure.values[0]
        c2_temp = ss[ss.atom_number==df_lc.iloc[i,1]].structure.values[0]
        c3_temp = ss[ss.atom_number==df_lc.iloc[i,2]].structure.values[0]

        c1.append(c1_temp)
        c2.append(c2_temp)
        c3.append(c3_temp)

    df_lc['ss_0'] = c1
    df_lc['ss_1'] = c2
    df_lc['ss_2'] = c3
    
    #columna con coordenadas del clique
    return(df_lc)

In [17]:
df_lc1 = get_SS(ss1,df_lc1)
df_lc2 = get_SS(ss2,df_lc2)

Hora de comparar los SS

In [18]:
def SSM(ss1,ss2):
    """Catalogo SSM siguiendo la tabla 1 y con una funcion extra,
    ss1: string (H,B,C)
    ss2: string (H,B,C)
    devuelve el score: int (0,1,2)"""
    def get_score_from_table(ss1,ss2):

        if (ss1 == 'H') and (ss2 == 'B'):
            score = 2
        elif (ss1 == 'B') and (ss2 == 'H'):
            score = 2
        else:
            print('WTF are u doing!')

        return(score)
    
    score = 123
    if ss1 ==  ss2:
        score = 0
    elif ((ss1 != ss2) & ((ss1 == 'C') | (ss2 == 'C'))):
        score = 1
    else:
        score = get_score_from_table(ss1,ss2)
        
    return(score)   

In [19]:
# producto = it.product(df_lc1.index.values,df_lc2.index.values)
cols = ['ss_0','ss_1','ss_2']
# for i in producto:
#     if i[0] < 1:
#         print(
#             list(#FUNCION PARA OBTENER LOS VALORES DE SSM
#                  map(SSM,df_lc1.iloc[i[0]][cols],df_lc2.iloc[i[1]][cols]))
#         )
#     else:
#         break

In [20]:
# df_lc1['score_ss'] =
a = df_lc1.iloc[0][cols]
b = df_lc2.iloc[-100][cols]
list(map(SSM,a,b))

[1, 1, 0]

In [21]:
df_lc1.head()

,0,1,2,ss_0,ss_1,ss_2
0,27,2,35,C,C,B
1,10,2,35,C,C,B
2,19,2,35,C,C,B
3,10,2,27,C,C,C
4,19,2,27,C,C,C


In [22]:
df_lc2.head()

,0,1,2,ss_0,ss_1,ss_2
0,129,161,139,H,H,H
1,129,161,173,H,H,H
2,152,129,161,H,H,H
3,184,129,161,H,H,H
4,129,161,193,H,H,H


In [23]:
# AGUAS CON EL OUTPUT NO LO DEJES ASI!!! HAY QUE QUITARLO
# for i in df_lc1['ss_0']:
#     for j in df_lc2['ss_0']:
# #         print(SSM(i,j))

In [24]:
for i in df_lc1.index:
    print(list(it.permutations(df_lc1[[0,1,2]].values[i],3)))
    break

[(27, 2, 35), (27, 35, 2), (2, 27, 35), (2, 35, 27), (35, 27, 2), (35, 2, 27)]


### DSSP A MANO 
sin puentes de hidrogeno solo por Ramachandran Plot y filtros feeling

__QUE SIEMPRE NO SE UTILIZA ESTE__

In [25]:
# #se genera un dataset con solo los atomos de interes para obtener la estructura
# df_dh1 = df_ca1[df_ca1.atom_name.isin(['N','CA','C',])].reset_index()
# df_dh2 = df_ca2[df_ca2.atom_name.isin(['N','CA','C',])].reset_index()

In [26]:
# def calculate_phi_psi(df_dh):
#     #calculo de Phi observar el orden que es C--N--CA--C
#     index_atom_number = []
#     # index_phi = []
#     angulos_phi = []
#     append = angulos_phi.append
#     valores = df_dh.vector.values
#     dihedral = mvt.dihedral
#     nombres = df_dh.atom_name
#     for i in range(df_dh.shape[0]-3):
#         if i == 0: # COMO NO TIENE CON QUIEN COMPARAR SE AGREGA QUE EL PRIMERO COMIENCE A 360 GRADOS
#             append(360.0)

#         elif (nombres[i] == 'C') and (nombres[i+1] == 'N') and (nombres[i+2] == 'CA') and (nombres[i+3] == 'C'):
#     #         index_phi.append(df_dh1.residue_number.values[i])
#             index_atom_number.append(df_dh.atom_number.values[i]-1)
#             append(dihedral(valores[i],valores[i+1],valores[i+2],valores[i+3]))

#     # index_phi.append(df_dh1.residue_number.values[-1])
#     index_atom_number.append(df_dh.atom_number.values[-1]-1)
    
#     #Calculo de psi con el orden N--CA--C--N
#     angulos_psi = []
#     append = angulos_psi.append
#     valores = df_dh.vector.values
#     dihedral = mvt.dihedral
#     nombres = df_dh.atom_name
#     for i in range(df_dh.shape[0]-3):
#             if (nombres[i] == 'N') and (nombres[i+1] == 'CA') and (nombres[i+2] == 'C') and (nombres[i+3] == 'N'):
#                 append(dihedral(valores[i],valores[i+1],valores[i+2],valores[i+3]))
                
#     angulos = pd.DataFrame([angulos_phi,angulos_psi]).T
#     angulos.columns = ['phi','psi']
#     angulos.phi = np.where(angulos.phi > 180, angulos.phi - 360, angulos.phi)
#     angulos.psi = np.where(angulos.psi > 180, angulos.psi - 360, angulos.psi)
#     angulos.replace( 0, 360, inplace = True)
#     angulos['atom_number'] = index_atom_number
#     angulos.fillna(360.0, inplace=True)
#     return(angulos)

In [27]:
# angulos1 = calculate_phi_psi(df_dh1)
# angulos2 = calculate_phi_psi(df_dh2)

In [28]:
# #Calculo de psi con el orden N--CA--C--N
# angulos_psi = []
# append = angulos_psi.append
# valores = df_dh1.vector.values
# dihedral = mvt.dihedral
# nombres = df_dh1.atom_name
# for i in range(df_dh1.shape[0]-3):
#         if (nombres[i] == 'N') and (nombres[i+1] == 'CA') and (nombres[i+2] == 'C') and (nombres[i+3] == 'N'):
#             append(dihedral(valores[i],valores[i+1],valores[i+2],valores[i+3]))

In [29]:
# SE REVISAN LOS ANGULOS QUE SEAN ADECUADOS
# SE CHECO UTILIZANDO DSSP ONLINE Y SI DAN LOS ANGULOS
#AHORA FALTA GENERAR O UN CATALOGO O EMPALMAR EL RAMACHANDRAN PLOT PARA
#OBTENER LA ESTRUCTURA SECUNDARIA
# angulos = pd.DataFrame([angulos_phi,angulos_psi]).T
# angulos.columns = ['phi','psi']
# angulos.phi = np.where(angulos.phi > 180, angulos.phi - 360, angulos.phi)
# angulos.psi = np.where(angulos.psi > 180, angulos.psi - 360, angulos.psi)
# angulos.replace( 0, 360, inplace = True)
# angulos['atom_number'] = index_atom_number
# angulos.fillna(360.0, inplace=True)

In [30]:
# siguiendo el catalogo de: https://www.researchgate.net/publication/220777003_Protein_Secondary_Structure_Prediction_Based_on_Ramachandran_Maps
#crearemos el catalogo para la SS
# para alfa-helice tienen que caer los puntos dentro del circulo de radio 7 con centro en (-63,-45)
# def inside_circle(x,y):
#     R = 7
#     x_center = -63.0
#     y_center = -45.0
#     distancia = np.sqrt((x - (x_center))**2 + (y - (y_center))**2)
#     return(distancia <= R)

# boolean_list_alfa_circle =  np.where(inside_circle(angulos.phi,angulos.psi),1,0)

In [31]:
# def inside_alfa_helix(x,y):
#     esta_en_x = -75.0 <= x <= -45.0
#     esta_en_y = -60.0 <= y <= -30.0
#     boolean = esta_en_x and esta_en_y
#     return(boolean)

# boolean_list_alfa = []
# for i in angulos.index:
#     resultado = inside_alfa_helix(angulos.phi[i],angulos.psi[i])
#     boolean_list_alfa.append(resultado)
    
# boolean_list_alfa = np.array(boolean_list_alfa) * 1

In [32]:
# def inside_beta_sheets(x,y):
#     esta_en_x = -180.0 <= x <= -105.0
#     esta_en_y = 120.0 <= y <= 180.0
#     boolean = esta_en_x and esta_en_y
#     return(boolean)

# boolean_list_beta = []
# for i in angulos.index:
#     resultado = inside_beta_sheets(angulos.phi[i],angulos.psi[i])
#     boolean_list_beta.append(resultado)
    
# boolean_list_beta = np.array(boolean_list_beta) * 1

In [33]:
# def secundary_structure(angulos,df_ca):
#     #alfa helices
#     def inside_circle(x,y):
#         R = 7
#         x_center = -63.0
#         y_center = -45.0
#         distancia = np.sqrt((x - (x_center))**2 + (y - (y_center))**2)
#         return(distancia <= R)

#     boolean_list_alfa_circle =  np.where(inside_circle(angulos.phi,angulos.psi),1,0)
#     #mas alfa helices
#     def inside_alfa_helix(x,y):
#         esta_en_x = -75.0 <= x <= -45.0
#         esta_en_y = -60.0 <= y <= -30.0
#         boolean = esta_en_x and esta_en_y
#         return(boolean)

#     boolean_list_alfa = []
#     for i in angulos.index:
#         resultado = inside_alfa_helix(angulos.phi[i],angulos.psi[i])
#         boolean_list_alfa.append(resultado)
    
#     boolean_list_alfa = np.array(boolean_list_alfa) * 1
#     #beta sheets
#     def inside_beta_sheets(x,y):
#         esta_en_x = -180.0 <= x <= -105.0
#         esta_en_y = 120.0 <= y <= 180.0
#         boolean = esta_en_x and esta_en_y
#         return(boolean)

#     boolean_list_beta = []
#     for i in angulos.index:
#         resultado = inside_beta_sheets(angulos.phi[i],angulos.psi[i])
#         boolean_list_beta.append(resultado)
    
#     boolean_list_beta = np.array(boolean_list_beta) * 1
#     #generacion de dataframe structure
#     structure = pd.DataFrame([boolean_list_alfa_circle,
#                               boolean_list_alfa,
#                               boolean_list_beta]).T
#     structure.columns = ['alfa_circulo','alfa_helice','beta_sheets']
#     structure['SS'] = np.where(structure.alfa_circulo == 1,'alfa_circulo',
#                               np.where(structure.alfa_helice == 1,'alfa_helice',
#                                  np.where(structure.beta_sheets == 1,'beta_sheets','COIL')))
#     angulos['SS'] = np.where(structure.SS.isin(['alfa_circulo','alfa_helice']),'H',
#                              np.where(structure.SS == 'beta_sheets','E','C'))
#     # # H -- ALFA HELIX
#     # # E -- BETA SHEETS
#     # # C -- COIL
# #     print(angulos.SS.value_counts(normalize = True) * 100)
#     df_ca = df_ca.merge(angulos[['atom_number','SS']], 
#                         how='left',on='atom_number')
#     return(df_ca)

In [34]:
# #calculo de SS y pegado
# df_ca1 = secundary_structure(angulos1,df_ca1)
# df_ca2 = secundary_structure(angulos2,df_ca2)

In [35]:
# #check de SS
# df_ca1.SS.value_counts(normalize=True)*100

In [36]:
# %%timeit
# secundary_structure(angulos1,df_ca1)

### Checks de Estructura secundaria
Para generar los checks correr el codigo sin funciones

In [37]:
# structure.SS.value_counts(normalize = True) * 100

In [38]:
# angulos['SS'] = np.where(structure.SS.isin(['alfa_circulo','alfa_helice']),'H',
#                          np.where(structure.SS == 'beta_sheets','E','C'))
# # # H -- ALFA HELIX
# # # E -- BETA SHEETS
# # # C -- COIL
# angulos.SS.value_counts(normalize = True) * 100

In [39]:
# angulos['color_SS'] = np.where(angulos.SS == 'H','r',
#                                np.where(angulos.SS == 'E','navy','g'))

# angulos[['atom_number','SS']]

In [40]:
# #Metodologia wiki
# #alfa helice (−90°, −15°) to (−35°, −70°) ROJO
# #betta (–135°, 135°) to (–180°, 180°) NAVY

In [41]:
# #RAMACHANDRAN PLOT SOLO ANGULOS
# angulos.plot.scatter('phi','psi', title='Ramachandran Plot', 
#                      c = angulos.color_SS.values.tolist(),
#                      marker = 'x',
#                      alpha=0.6, figsize=(10,10), s=80
#                     )
# limite1,limite2 = -190,190
# plt.xlim(limite1,limite2)
# plt.ylim(limite1,limite2);

In [42]:
#funcion para obtener las coordenadas del clique
def get_coord_clique(df_ca,df_lc):
    """df_ca:DataFrame con coordenadas de carbonos alfa,
    df_lc:Dataframe con cliques, si coincide el numero del atomo
    le pega su coordenada y genera una matriz de vectores que contiene 
    las coordenadas de cada atomo ordenado de izquierda a derecha como 
    aparecen en df_lc"""
    lista_matriz_coordendas = [] #lista para apilar las coordenadas
    x = []
    y = []
    z = []

    for i in df_lc.index:
        #si coincide el numero de atomo con el numero de atomo del clique le coloca el vector de coordenadas
        x_temp = np.array(df_ca[df_ca.atom_number==df_lc.iloc[i,0]].vector.values[0])
        y_temp = np.array(df_ca[df_ca.atom_number==df_lc.iloc[i,1]].vector.values[0])
        z_temp = np.array(df_ca[df_ca.atom_number==df_lc.iloc[i,2]].vector.values[0])
        mat_dist = [x_temp,y_temp,z_temp]

        x.append(x_temp)
        y.append(y_temp)
        z.append(z_temp)
        lista_matriz_coordendas.append(mat_dist)

    df_lc['coord_clique_0'] = x
    df_lc['coord_clique_1'] = y
    df_lc['coord_clique_2'] = z
    df_lc['matriz_coordenadas'] = lista_matriz_coordendas #columna con coordenadas del clique
    return(df_lc)

In [43]:
#pegado de coordendas
df_lc1 = get_coord_clique(df_ca1,df_lc1)
df_lc2 = get_coord_clique(df_ca2,df_lc2)

## Comparacion de cliques
Para obtener el __RMSD__ es necesario primero rotar y trasladar un atomo con respecto al atomo a comparar (de la otra proteina) y calcular el __RMSD__.

Siguiendo al metodologia en *Using quaternions to calculate RMSD*.
Se generan las funciones de traslado y rotacion.

### Traslacion
Se calcula el baricentro de cada clique en ambas moleculas y se generan nuevos vectores que van del baricentro al atomo llamados $\hat{x}$.

El baricentro se calcula como $\bar{x} =$($\frac{(x_1 + x_2 + x_3)}{3}$,$\frac{(y_1 + y_2 + y_3)}{3}$,$\frac{(z_1 + z_2 + z_3)}{3}$)

$\hat{x} = x_k - \bar{x}$

In [44]:
num_cliques = 3

A = df_lc1.coord_clique_0.values
B = df_lc1.coord_clique_1.values
C = df_lc1.coord_clique_2.values
D = np.zeros((1, num_cliques))
E = np.zeros((1, num_cliques))
F = np.zeros((1, num_cliques))
G = np.zeros((1, num_cliques))
if num_cliques >= 4:
    D = df_lc1.coord_clique_3.values.reshape(-1, 1)
if num_cliques >= 5:
    E = df_lc1.coord_clique_4.values.reshape(-1, 1)
if num_cliques >= 6:
    F = df_lc1.coord_clique_5.values.reshape(-1, 1)
if num_cliques >= 7:
    G = df_lc1.coord_clique_6.values.reshape(-1, 1)

print()    
# for i in range(df_lc1.shape[0]):
    
# print(df_cliques.coord_clique_0[0])
# algo = [a[0] for a in A]
# algo2 = [a[0] for a in B]
# algo3 = [a[0] for a in C]

# print(algo[0],algo2[0],algo3[0])
# x1 = np.divide(np.sum([algo,algo2,algo3],0),3)
# x1

# x1 = np.sum([A[:, 0], B[:, 0], C[:, 0]], 0) / num_cliques
# print(x1)
# if num_cliques == 3:
#     coord_center = []
#     for i in range(df_cliques.shape[0]):
#         # se extrae las coordenadas de los atomos
#         A = df_cliques.coord_clique_0[i]
#         B = df_cliques.coord_clique_1[i]
#         C = df_cliques.coord_clique_2[i]
#         # se calcula el punto promedio
#         x1 = (A[0]+B[0]+C[0])/num_cliques
#         y1 = (A[1]+B[1]+C[1])/num_cliques
#         z1 = (A[2]+B[2]+C[2])/num_cliques
#         # se apila para pegarlo en una sola fila correspondiente al clique
#         coord_center.append(np.array([x1, y1, z1]))
#
#     #generacion de la columna
#     df_cliques['baricentro_clique'] = coord_center

In [45]:
df_lc1.head()

,0,1,2,ss_0,ss_1,ss_2,coord_clique_0,coord_clique_1,coord_clique_2,matriz_coordenadas
0,27,2,35,C,C,B,"[22.929, 57.825, 21.316]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[22.929, 57.825, 21.316], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]"
1,10,2,35,C,C,B,"[27.877, 57.84, 21.376]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[27.877, 57.84, 21.376], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]"
2,19,2,35,C,C,B,"[25.404, 54.999, 20.779]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[25.404, 54.999, 20.779], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]"
3,10,2,27,C,C,C,"[27.877, 57.84, 21.376]","[28.952, 56.593, 24.813]","[22.929, 57.825, 21.316]","[[27.877, 57.84, 21.376], [28.952, 56.593, 24.813], [22.929, 57.825, 21.316]]"
4,19,2,27,C,C,C,"[25.404, 54.999, 20.779]","[28.952, 56.593, 24.813]","[22.929, 57.825, 21.316]","[[25.404, 54.999, 20.779], [28.952, 56.593, 24.813], [22.929, 57.825, 21.316]]"


In [46]:
# funcion de calculo de baricentro
def baricenter_clique(df_lc):
    """se calcula el baricentro de cada clique 
    siguiendo la formula de arriba.
    df_lc: Dataframe con los cliques y coordenadas
    regresa
    df_lc:Dataframe con el baricentro de ese clique"""
    coord_center = []
    for i in range(df_lc.shape[0]):
        #se extrae las coordenadas de los atomos
        A = df_lc.coord_clique_0[i]
        B = df_lc.coord_clique_1[i]
        C = df_lc.coord_clique_2[i]
        #se calcula el punto promedio
        x1 = round((A[0]+B[0]+C[0])/3,5)
        y1 = round((A[1]+B[1]+C[1])/3,5)
        z1 = round((A[2]+B[2]+C[2])/3,5)
        #se apila para pegarlo en una sola fila correspondiente al clique
        coord_center.append(np.array([x1,y1,z1]))

    #generacion de la columna
    df_lc['baricentro_clique'] = coord_center
    return(df_lc)

In [47]:
baricenter_clique(df_lc1).head()

,0,1,2,ss_0,ss_1,ss_2,coord_clique_0,coord_clique_1,coord_clique_2,matriz_coordenadas,baricentro_clique
0,27,2,35,C,C,B,"[22.929, 57.825, 21.316]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[22.929, 57.825, 21.316], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]","[25.17, 57.903, 21.33167]"
1,10,2,35,C,C,B,"[27.877, 57.84, 21.376]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[27.877, 57.84, 21.376], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]","[26.81933, 57.908, 21.35167]"
2,19,2,35,C,C,B,"[25.404, 54.999, 20.779]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[25.404, 54.999, 20.779], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]","[25.995, 56.961, 21.15267]"
3,10,2,27,C,C,C,"[27.877, 57.84, 21.376]","[28.952, 56.593, 24.813]","[22.929, 57.825, 21.316]","[[27.877, 57.84, 21.376], [28.952, 56.593, 24.813], [22.929, 57.825, 21.316]]","[26.586, 57.41933, 22.50167]"
4,19,2,27,C,C,C,"[25.404, 54.999, 20.779]","[28.952, 56.593, 24.813]","[22.929, 57.825, 21.316]","[[25.404, 54.999, 20.779], [28.952, 56.593, 24.813], [22.929, 57.825, 21.316]]","[25.76167, 56.47233, 22.30267]"


In [48]:
#calculo de baricentro
df_lc1 = baricenter_clique(df_lc1)
df_lc2 = baricenter_clique(df_lc2)

In [49]:
def center_vectors(df_lc):
    """Calculo de los vectores gorro que van del baricentro 
    a la coordenada del atomo
    df_lc: Dataframe con baricentro y coordenadas de cada clique
    regresa
    df_lc:Dataframe con vectores gorro en otra columna"""
    vec1 = []
    vec2 = []
    vec3 = []
    vectores_centricos = []
    for i,val in enumerate(df_lc.baricentro_clique):
    #     extraccion de coordenadas de cada atomo
        A = df_lc.coord_clique_0[i]
        B = df_lc.coord_clique_1[i]
        C = df_lc.coord_clique_2[i]
        #calculo de vectores DEL CENTRO AL PUNTO COORDENADA
        vec_a = np.round(list(A - val),6)
        vec_b = np.round(list(B - val),6)
        vec_c = np.round(list(C - val),6)
    #SE APILAN PARA QUE ESTEN EN EL MISMO CLIQUE CORRESPONDIENTE A CADA UNO.
        vec1.append(vec_a)
        vec2.append(vec_b)
        vec3.append(vec_c)
        vectores_centricos.append(np.array([vec_a,vec_b,vec_c]))
    #se generan la columna de cada vector correspondiente a cada atomo
    df_lc['vec_gorro_0'] = vec1
    df_lc['vec_gorro_1'] = vec2
    df_lc['vec_gorro_2'] = vec3
    df_lc['vectores_gorro'] = vectores_centricos
    return(df_lc)

In [50]:
#generacion de vectores gorro
df_lc1 = center_vectors(df_lc1)
df_lc2 = center_vectors(df_lc2)

### Rotacion
Para generar la rotacion tenemos que generar la *matriz gigante* que depende de los elemento de la matriz de correlacion $R_{ij}$

Donde $R_{ij} = \sum\limits_{k=1}^N{x_{ik}y_{jk}}, i,j = 1,2,3$

Posteriormente se calculan los eigenvalores y eigenvectores de esta matriz gigante
Para obtener los quaterniones y generar la matriz de rotacion y con ella calcular el vector rotado

Por ultimo, se suma al vector rotado y trasladado se suma el baricentro del clique a comparar y se calcula el RMSD

In [51]:
# No conviene utilizar pandas ya que tarda al acceder a los datos, buscar la manera
#usual para acceder a los datos y seguir avanzando
prueba1 = df_lc1.values
prueba2 = df_lc2.values

In [52]:
df_lc1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1787 entries, 0 to 1786
Data columns (total 15 columns):
0                     1787 non-null int64
1                     1787 non-null int64
2                     1787 non-null int64
ss_0                  1787 non-null object
ss_1                  1787 non-null object
ss_2                  1787 non-null object
coord_clique_0        1787 non-null object
coord_clique_1        1787 non-null object
coord_clique_2        1787 non-null object
matriz_coordenadas    1787 non-null object
baricentro_clique     1787 non-null object
vec_gorro_0           1787 non-null object
vec_gorro_1           1787 non-null object
vec_gorro_2           1787 non-null object
vectores_gorro        1787 non-null object
dtypes: int64(3), object(12)
memory usage: 209.5+ KB


In [53]:
df_lc1.head()

,0,1,2,ss_0,ss_1,ss_2,coord_clique_0,coord_clique_1,coord_clique_2,matriz_coordenadas,baricentro_clique,vec_gorro_0,vec_gorro_1,vec_gorro_2,vectores_gorro
0,27,2,35,C,C,B,"[22.929, 57.825, 21.316]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[22.929, 57.825, 21.316], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]","[25.17, 57.903, 21.33167]","[-2.241, -0.078, -0.01567]","[3.782, -1.31, 3.48133]","[-1.541, 1.388, -3.46567]","[[-2.241, -0.078, -0.01567], [3.782, -1.31, 3.48133], [-1.541, 1.388, -3.46567]]"
1,10,2,35,C,C,B,"[27.877, 57.84, 21.376]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[27.877, 57.84, 21.376], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]","[26.81933, 57.908, 21.35167]","[1.05767, -0.068, 0.02433]","[2.13267, -1.315, 3.46133]","[-3.19033, 1.383, -3.48567]","[[1.05767, -0.068, 0.02433], [2.13267, -1.315, 3.46133], [-3.19033, 1.383, -3.48567]]"
2,19,2,35,C,C,B,"[25.404, 54.999, 20.779]","[28.952, 56.593, 24.813]","[23.629, 59.291, 17.866]","[[25.404, 54.999, 20.779], [28.952, 56.593, 24.813], [23.629, 59.291, 17.866]]","[25.995, 56.961, 21.15267]","[-0.591, -1.962, -0.37367]","[2.957, -0.368, 3.66033]","[-2.366, 2.33, -3.28667]","[[-0.591, -1.962, -0.37367], [2.957, -0.368, 3.66033], [-2.366, 2.33, -3.28667]]"
3,10,2,27,C,C,C,"[27.877, 57.84, 21.376]","[28.952, 56.593, 24.813]","[22.929, 57.825, 21.316]","[[27.877, 57.84, 21.376], [28.952, 56.593, 24.813], [22.929, 57.825, 21.316]]","[26.586, 57.41933, 22.50167]","[1.291, 0.42067, -1.12567]","[2.366, -0.82633, 2.31133]","[-3.657, 0.40567, -1.18567]","[[1.291, 0.42067, -1.12567], [2.366, -0.82633, 2.31133], [-3.657, 0.40567, -1.18567]]"
4,19,2,27,C,C,C,"[25.404, 54.999, 20.779]","[28.952, 56.593, 24.813]","[22.929, 57.825, 21.316]","[[25.404, 54.999, 20.779], [28.952, 56.593, 24.813], [22.929, 57.825, 21.316]]","[25.76167, 56.47233, 22.30267]","[-0.35767, -1.47333, -1.52367]","[3.19033, 0.12067, 2.51033]","[-2.83267, 1.35267, -0.98667]","[[-0.35767, -1.47333, -1.52367], [3.19033, 0.12067, 2.51033], [-2.83267, 1.35267, -0.98667]]"


In [54]:
prueba1

array([[27, 2, 35, ..., array([ 3.782  , -1.31   ,  3.48133]),
        array([-1.541  ,  1.388  , -3.46567]),
        array([[-2.241  , -0.078  , -0.01567],
       [ 3.782  , -1.31   ,  3.48133],
       [-1.541  ,  1.388  , -3.46567]])],
       [10, 2, 35, ..., array([ 2.13267, -1.315  ,  3.46133]),
        array([-3.19033,  1.383  , -3.48567]),
        array([[ 1.05767, -0.068  ,  0.02433],
       [ 2.13267, -1.315  ,  3.46133],
       [-3.19033,  1.383  , -3.48567]])],
       [19, 2, 35, ..., array([ 2.957  , -0.368  ,  3.66033]),
        array([-2.366  ,  2.33   , -3.28667]),
        array([[-0.591  , -1.962  , -0.37367],
       [ 2.957  , -0.368  ,  3.66033],
       [-2.366  ,  2.33   , -3.28667]])],
       ...,
       [128, 205, 365, ..., array([2.32133, 0.62167, 4.99733]),
        array([-2.00467,  3.03667, -2.54767]),
        array([[-0.31667, -3.65833, -2.44967],
       [ 2.32133,  0.62167,  4.99733],
       [-2.00467,  3.03667, -2.54767]])],
       [128, 376, 205, ..., array([

In [55]:
for i,val in enumerate(df_lc1.columns):
    print(i,val)

0 0
1 1
2 2
3 ss_0
4 ss_1
5 ss_2
6 coord_clique_0
7 coord_clique_1
8 coord_clique_2
9 matriz_coordenadas
10 baricentro_clique
11 vec_gorro_0
12 vec_gorro_1
13 vec_gorro_2
14 vectores_gorro


In [56]:
%load_ext Cython

In [57]:

#funcion para obtener los valores de la prerotacion, de los valores de la matriz de correlaciones
# en check por que se utiliza vector gorro en lugar de posiciones iniciales 
# el articulo no dice...

def R_ij(i,j,a1=0,a2=0):
    """Recuerda que 0-->1,1-->2,2-->2 en los indices de R
    a1,a2 corresponden a que atomo quieren que se compare 
    """
    #se genera un diccionario para asignar los valores como en el articulo 
    #y no tener equivocaciones
    #     dict_convencion = {1:0,2:1,3:2}

    #     i = dict_convencion.get(i)
    #     j = dict_convencion.get(j)

    #     values = []
    #     append = values.append
    #     for k in [11,12,13]: #8,9,10 corresponde a la columna de vec_gorro_0,_1,_2 
    #         #REVISAR VEC_GORRO
    #         atom_value1 = prueba1[:,k][a1][i]
    #         atom_value2 = prueba2[:,k][a2][j]
    #         value = atom_value1 * atom_value2
    #         append(value)

    #     valor = sum(values)

    valor = sum([prueba1[:,k][a1][i] * prueba2[:,k][a2][j] for k in [11,12,13]])
    return(valor)

In [58]:
R_ij(0,0)

-2.013687

In [59]:
%%timeit
R_ij(1,1)

2.83 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [60]:
def giant_matrix(i,j):
    """cliques a comparar: i,j
    desde aqui se itera sobre cada i y hay que variar los vectores 
    coordenada 
    Regresa la matriz gigante (matriz simetrica del articulo)"""
    #primer renglon
    R11R22R33 = (R_ij(0,0,a1=i,a2=j) + R_ij(1,1,a1=i,a2=j) + R_ij(2,2,a1=i,a2=j))
    R23_R32 = (R_ij(1,2,a1=i,a2=j) - R_ij(2,1,a1=i,a2=j))
    R31_R13 = (R_ij(2,0,a1=i,a2=j) - R_ij(0,2,a1=i,a2=j))
    R12_R21 = (R_ij(0,1,a1=i,a2=j) - R_ij(1,0,a1=i,a2=j))
    #segundo renglon
    R11_R22_R33 = (R_ij(0,0,a1=i,a2=j) - R_ij(1,1,a1=i,a2=j) - R_ij(2,2,a1=i,a2=j))
    R12R21 = (R_ij(0,1,a1=i,a2=j) + R_ij(1,0,a1=i,a2=j))
    R13R31 = (R_ij(0,2,a1=i,a2=j) + R_ij(2,0,a1=i,a2=j))
    #tercer renglon
    _R11R22_R33 = (-R_ij(0,0,a1=i,a2=j) + R_ij(1,1,a1=i,a2=j) - R_ij(2,2,a1=i,a2=j))
    R23R32 = (R_ij(1,2,a1=i,a2=j) + R_ij(2,1,a1=0,a2=0))
    #cuarto renglon
    _R11_R22R33 = (-R_ij(0,0,a1=i,a2=j) - R_ij(1,1,a1=i,a2=j) + R_ij(2,2,a1=i,a2=j))

    matriz_gigante = [
        [R11R22R33, R23_R32 , R31_R13, R12_R21],
        [R23_R32, R11_R22_R33, R12R21, R13R31],
        [R31_R13, R12R21, _R11R22_R33, R23R32],
        [R12_R21, R13R31, R23R32, _R11_R22R33]
    ]

    return(matriz_gigante)

In [61]:
giant_matrix(1,1)

[[-11.714485999999999,
  -3.7932403332999995,
  -9.270034999999998,
  -1.8755423332999994],
 [-3.7932403332999995, -12.985943999999998, 8.315785666699998, -16.221443],
 [-9.270034999999998, 8.315785666699998, 9.0648, -8.919198],
 [-1.8755423332999994, -16.221443, -8.919198, 15.635629999999997]]

In [62]:
%%timeit
giant_matrix(1,1)

69.3 µs ± 716 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [63]:
def rotation_matrix(matriz_gigante):
    """utilizando la funcion giant_matrix, fijando los valores de i,j
    se calcula la matriz de rotacion con los eigenvectores y eigenvalores
    arroja una matriz de rotacion que depende de la matriz gigante
    """
    eignvalues,eigenvectors = np.linalg.eig(matriz_gigante)
    q = eigenvectors[:,np.argmax(eignvalues)]
    q0,q1,q2,q3 = q[0],q[1],q[2],q[3]
    #matriz de rotacion con eigenvectores
    mat_rot = np.array([
                [(q0**2+q1**2-q2**2-q3**2), 2*(q1*q2-q0*q3),2*(q1*q3+q0*q2)],
                [2*(q1*q2+q0*q3), (q0**2-q1**2+q2**2-q3**2),2*(q2*q3-q0*q1)],
                [2*(q1*q3-q0*q2),2*(q2*q3+q0*q1), (q0**2-q1**2-q2**2+q3**2)]
    ], dtype=np.float64)
    return(mat_rot)

In [64]:
rotation_matrix(giant_matrix(1,1))

array([[-0.66396017,  0.23453208, -0.71003634],
       [ 0.58745962, -0.42389083, -0.68935315],
       [-0.46265332, -0.87482071,  0.1436685 ]])

In [65]:
%%timeit
rotation_matrix(giant_matrix(1,1))

128 µs ± 5.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [66]:
def rotation_vectors(vector_gorro,mat_rot):
    """obtencion de vector rotado,
    utilizando la matriz de rotacion 
    y los vectores gorro a rotar y trasladar"""
    #multiplicacion de matrices de cada vector rotado
#     coord_rot_tras = []
#     append = coord_rot_tras.append
#     matmul = np.matmul
#     for i in vector_gorro:
#         append(matmul(mat_rot,i.reshape(3,1)).T[0])

    coord_rot_tras = [np.matmul(mat_rot,i.reshape(3,1)).T[0] for i in vector_gorro]
    return(coord_rot_tras)

In [67]:
matriz_gigante = giant_matrix(1,1)
mat_rot = rotation_matrix(matriz_gigante)
x_rot = rotation_vectors(prueba1[:,14][1],mat_rot) #vector_gorro vs matriz_rotacion
x_rot

[array([-0.73547411,  0.63339103, -0.42635127]),
 array([-4.18208769, -0.57580477,  0.66098647]),
 array([ 4.91756226, -0.05757349, -0.23464126])]

In [68]:
%%timeit
matriz_gigante = giant_matrix(1,1)
mat_rot = rotation_matrix(matriz_gigante)
x_rot = rotation_vectors(prueba1[:,14][1],mat_rot) #vector_gorro vs matriz_rotacion
x_rot

129 µs ± 512 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Calculo del RMSD con previo filtro de SSM y SAM
Aqui iria el codigo de SSM y SAM para filtrado y despues se calcula el rmsd de cada clique, por lo que primero hay que filtrar

In [69]:
def rmsd_between_cliques(atom_trans_rot,atom_to_compare):
    """Calculo de rmsd entre cliques tomando el atomo rotado y trasladado
    y el atomo a comparar, por el momento solo imprime el resultado"""
    # primer RMSD entre atomos
    p12 = np.sum((np.array(atom_to_compare, dtype=np.float64)-atom_trans_rot)**2,1)
    rmsd_i = lambda i: np.sqrt(i)/3
    rmsd_final = rmsd_i(p12).mean()
    
    if rmsd_final <= 0.15: ##AQUI LOS DETECTA QUIENES CUMPLEN CON EL FILTRO...
        print('RMSD_final:', rmsd_final)
    
    return(rmsd_final)

In [70]:
matriz_gigante = giant_matrix(1,1)
mat_rot = rotation_matrix(matriz_gigante)
x_rot = rotation_vectors(prueba1[:,14][1],mat_rot)
coord_rot_clique_2 = x_rot + np.array(prueba2[:,10][1], dtype=np.float64)
print(rmsd_between_cliques(coord_rot_clique_2,np.array(prueba2[:,9][1], dtype=np.float64)))

1.0272843125738795


In [71]:
prueba2[:,9][1]

[array([13.454,  5.239, -2.835]),
 array([14.115,  5.057,  2.255]),
 array([17.767,  4.108,  1.657])]

In [72]:
def calculate_rmsd_rot_trans(atom1,atom2):
    matriz_gigante = giant_matrix(atom1,atom2)
    mat_rot = rotation_matrix(matriz_gigante)
    x_rot = rotation_vectors(prueba1[:,14][atom1],mat_rot)
    coord_rot_clique_2 = x_rot + np.array(prueba2[:,10][atom2], dtype=np.float64) #xrot + baricentro a mover
    rmsd_between_cliques(coord_rot_clique_2,np.array(prueba2[:,9][atom2], dtype=np.float64))
    # clique rotado y trasladado vs clique coordenadas

In [73]:
# for i in range(df_lc2.shape[0]): 
#     print(calculate_rmsd_rot_trans(10,i))

In [74]:
%%prun
for i in range(df_lc2.shape[0]): 
    calculate_rmsd_rot_trans(100,i)

RMSD_final: 0.13865310323512825
RMSD_final: 0.12205866995897925
RMSD_final: 0.11743363574558079
RMSD_final: 0.07172511570984799
RMSD_final: 0.09638199527572511
RMSD_final: 0.13233109117819855
 

         294624 function calls (294623 primitive calls) in 0.465 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    50448    0.120    0.000    0.120    0.000 <ipython-input-57-b52d72c3869c>:28(<listcomp>)
     2102    0.061    0.000    0.119    0.000 linalg.py:1140(eig)
     8408    0.030    0.000    0.030    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    12612    0.029    0.000    0.029    0.000 {built-in method numpy.core.multiarray.array}
     2102    0.027    0.000    0.461    0.000 <ipython-input-72-a1c5cce76344>:1(calculate_rmsd_rot_trans)
    50448    0.023    0.000    0.161    0.000 <ipython-input-57-b52d72c3869c>:6(R_ij)
     2102    0.021    0.000    0.157    0.000 <ipython-input-63-a78f53ea61d0>:1(rotation_matrix)
     2102    0.018    0.000    0.179    0.000 <ipython-input-60-4569b05293a5>:1(giant_matrix)
    50448    0.017    0.000    0.017    0.000 {built-in method builtins.sum}
     2102    0.012    0.

In [75]:
%%time
# for j in range(df_lc1.shape[0]):
#     for i in range(df_lc2.shape[0]):
#         calculate_rmsd_rot_trans(j,i)

# %%time
#para quitarme el for anidado puedo hacerlo con este producto por lo que reduce ligeramente pero aun no se puede medir
# #dado que son muchas operaciones 13000 * 2000
# producto = it.product(df_lc1.index.values,df_lc2.index.values)
# for i,j in producto:
#     calculate_rmsd_rot_trans(j,i)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [76]:
producto = it.product(df_lc1.index.values,df_lc2.index.values)

In [77]:
# list(producto)